**torch.autograd** Autograd là công cụ tự động tính đạo hàm của PyTorch, hỗ trợ huấn luyện mạng nơ-ron. Trong phần này, bạn sẽ hiểu được khái niệm về cách Autograd giúp huấn luyện mạng nơ-ron.

Hãy cùng xem xét một bước huấn luyện đơn lẻ. Trong ví dụ này, chúng ta tải một mô hình resnet18 được huấn luyện trước từ torchvision. Chúng ta tạo một tensor dữ liệu ngẫu nhiên để biểu diễn một hình ảnh duy nhất với 3 kênh, chiều cao và chiều rộng là 64, và được label khởi tạo tương ứng với một số giá trị ngẫu nhiên. Nhãn trong các mô hình được huấn luyện trước có hình dạng (1,1000).

In [1]:
import torch
from torchvision.models import resnet18, ResNet18_Weights
model = resnet18(weights=ResNet18_Weights.DEFAULT)
data = torch.rand(1, 3, 64, 64)
labels = torch.rand(1, 1000)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 217MB/s]


Tiếp theo, chúng ta chạy dữ liệu đầu vào qua mô hình ở từng lớp của nó để đưa ra dự đoán. Đây là bước truyền tiến (forward pass ).

In [2]:
prediction = model(data) # forward pass

Chúng ta sử dụng dự đoán của mô hình và nhãn tương ứng để tính toán lỗi (loss). Bước tiếp theo là lan truyền ngược lỗi này qua mạng. Quá trình lan truyền ngược được bắt đầu khi chúng ta gọi .backward() tensor lỗi. Autograd sau đó tính toán và lưu trữ độ dốc cho mỗi tham số mô hình trong  huộc tính của tham số .grad.

In [3]:
loss = (prediction - labels).sum()
loss.backward() # backward pass

Tiếp theo, chúng ta tải một trình tối ưu hóa, trong trường hợp này là SGD với tốc độ học là 0.01 và động lượng là 0.9. 
Có thể hiểu động lượng ở đây là quán tính của đạo hàm trước đó tức là cộng với 0.9 lần đạo hàm trước đó vào đạo hàm hiện tại, giúp cho gradient đi nhanh hơn nếu có cùng hướng và làm giảm độ dao động giữa cacs bước gradient khác hướng

In [4]:
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

Cuối cùng, chúng ta gọi hàm .step() để bắt đầu thuật toán tối ưu hóa gradient descent. Thuật toán tối ưu hóa sẽ điều chỉnh từng tham số dựa trên độ dốc của nó được lưu trữ trong biến .grad.

In [5]:
optim.step() #gradient descent

Trong mạng nơ-ron (NN), các tham số không tính toán đạo hàm thường được gọi là các tham số đóng băng . Việc "đóng băng" một phần mô hình rất hữu ích nếu bạn biết trước rằng mình sẽ không cần đến đạo hàm của các tham số đó (điều này mang lại một số lợi ích về hiệu suất bằng cách giảm số lần tính toán tự động đạo hàm).

Trong quá trình tinh chỉnh, chúng ta đóng băng hầu hết mô hình và thường chỉ sửa đổi các lớp phân loại để đưa ra dự đoán trên các nhãn mới. Hãy cùng xem qua một ví dụ nhỏ để minh họa điều này. Như trước đây, chúng ta tải một mô hình resnet18 đã được huấn luyện trước và đóng băng tất cả các tham số.

In [ ]:
from torch import nn, optim

model = resnet18(weights=ResNet18_Weights.DEFAULT)

# Freeze all the parameters in the network
for param in model.parameters():
    param.requires_grad = False

Giả sử chúng ta muốn tinh chỉnh mô hình trên một tập dữ liệu mới với 10 nhãn. Trong ResNet, lớp phân loại là lớp tuyến tính cuối cùng model.fc. Chúng ta có thể đơn giản thay thế nó bằng một lớp tuyến tính mới (mặc định không bị đóng băng) hoạt động như lớp phân loại của chúng ta.

In [ ]:
model.fc = nn.Linear(512, 10)

Giờ đây, tất cả các tham số trong mô hình, ngoại trừ các tham số của model.fc, đều được cố định. Các tham số duy nhất tính toán độ dốc là trọng số và độ lệch của model.fc.